In [ ]:
import numpy as np
from evaluation import evaluate_all
from solution import *
from MatrixVectorizer import MatrixVectorizer
import pandas as pd
import torch

from preprocessing import unpad

def main():
    random_seed = 42
    np.random.seed(random_seed)

    device = torch.device('cpu')

    IN_DIM = 160
    OUT_DIM = 268


    # Initialize vectorizer
    mv = MatrixVectorizer()

    # Load the data

    # Load the data
    fold1_lr = pd.read_csv("data/randomCV/Split1/lr_train_split_1.csv", header=None, skiprows=1)
    fold1_hr = pd.read_csv("data/randomCV/Split1/hr_train_split_1.csv", header=None, skiprows=1)
    fold2_lr = pd.read_csv("data/randomCV/Split2/lr_train_split_2.csv", header=None, skiprows=1)
    fold2_hr = pd.read_csv("data/randomCV/Split2/hr_train_split_2.csv", header=None, skiprows=1)
    fold3_lr = pd.read_csv("data/randomCV/Split3/lr_train_split_3.csv", header=None, skiprows=1)
    fold3_hr = pd.read_csv("data/randomCV/Split3/hr_train_split_3.csv", header=None, skiprows=1)

    # convert dataframes to numpy arrays
    fold1_lr = fold1_lr.to_numpy()
    fold1_hr = fold1_hr.to_numpy()
    fold2_lr = fold2_lr.to_numpy()
    fold2_hr = fold2_hr.to_numpy()
    fold3_lr = fold3_lr.to_numpy()
    fold3_hr = fold3_hr.to_numpy()

    # Store the fold results
    fold_results = []

    # Perform cross-validation
    for current_fold in range(1, 4):
        print(f"Fold {current_fold}: ")

        # Load the training and testing data for the current fold
        if current_fold == 1:
            low_res_test = fold1_lr
            high_res_test = fold1_hr
            low_res_train = np.concatenate((fold2_lr, fold3_lr))
            high_res_train = np.concatenate((fold2_hr, fold3_hr))

        elif current_fold == 2:
            low_res_test = fold2_lr
            high_res_test = fold2_hr
            low_res_train = np.concatenate((fold1_lr, fold3_lr))
            high_res_train = np.concatenate((fold1_hr, fold3_hr))

        else:
            low_res_test = fold3_lr
            high_res_test = fold3_hr
            low_res_train = np.concatenate((fold1_lr, fold2_lr))
            high_res_train = np.concatenate((fold1_hr, fold2_hr))



        # Vectorize the training and testing data
        lr_train_tensor = torch.stack([torch.from_numpy(mv.anti_vectorize(low_res_train[lr], IN_DIM)) for lr in range(low_res_train.shape[0])]).float()
        hr_train_tensor = torch.stack([torch.from_numpy(mv.anti_vectorize(high_res_train[lr], OUT_DIM)) for lr in range(high_res_train.shape[0])]).float()

        lr_test_tensor = torch.stack([torch.from_numpy(mv.anti_vectorize(low_res_test[lr], IN_DIM)) for lr in range(low_res_test.shape[0])]).float()
        high_res_test = np.array([mv.anti_vectorize(high_res_test[lr], OUT_DIM) for lr in range(high_res_test.shape[0])])

        print(f"Train input matrices shape: {lr_train_tensor.shape}")
        print(f"Train output matrices shape: {hr_train_tensor.shape}")

        print(f"Test input matrices shape: {lr_test_tensor.shape}")
        print(f"Test output matrices shape: {high_res_test.shape}")



        # Train the model
        random_seed = 42

        config = {
            'epochs': 22,
            'lambda': 16,
            'device' : device,
            'padding': 26,
            'lr_dim': IN_DIM,
            'hr_dim': 2*IN_DIM,
            'hidden_dim': 320,
            'drop_p': 0.5,
            'lr': 1e-3,
            'random_seed': random_seed,
            'gsr_layers': [0.9, 0.8, 0.6, 0.5],
            'permutate_graph': False,
            'adversarial': True,
            "data_augment": False
        }

        fold_model = train_model(lr_train_tensor, hr_train_tensor, config, lr_val=None, hr_val=None, fold=current_fold)

        predictions = []

        for i in range(lr_test_tensor.shape[0]):
            hr_pred = unpad(fold_model(lr_test_tensor[i])[0].detach().fill_diagonal_(0).numpy(), 26)
            predictions.append(hr_pred)
        
        predictions = np.array(predictions)
        # Evaluate the model on the test set and log the results
        # convert test_input_matrices to tensor transposed
        metrics = evaluate_all(
            high_res_test, predictions, output_path=f'results/randomCV_fold{current_fold}.csv'
        )
        fold_results.append(metrics)

if __name__ == '__main__':
    main()